In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Runnable


# Runnable 구조 검토

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

In [5]:
# Vector Store 생성
vector_store = FAISS.from_texts(
    [
        "영희는 스칼라 주식회사에서 근무하고 있습니다.",
        "철수는 같은 회사에서 근무 하였습니다.",
        "영희 직업은 개발자 입니다.",
        "철수의 직업은 컨텐츠 기획자 입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)

# Vector Store를 검색기로 사용
retriever = vector_store.as_retriever()

# 템플릿 정의
template = """
Answer the question based only on the following context: {context}
Question: {question}
"""

# Prompt 정의
prompt = PromptTemplate.from_template(template)

# 모델 정의/초기화
model = ChatOpenAI(model_name="gpt-4o-mini")


# 문서 포맷 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# Chain 구성
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 그래프 구성 확인

`chain.get_graph()` 
- 체인의 각 노드와 노드 간의 연결을 나타내는 그래프 객체 반환
- Node : 체인의 각 단계
- Edge : 단계 간의 데이터 흐름

In [ ]:
# 그래프를 그리기 위한 라이브러리 설치
# !pip install -qU grandalf

In [8]:
retrieval_chain.get_graph().nodes

{'4b083e49e9ae4ac597161f0750d82700': Node(id='4b083e49e9ae4ac597161f0750d82700', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None),
 '6ae2e26704cd4f358fd144c466a631cc': Node(id='6ae2e26704cd4f358fd144c466a631cc', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None),
 '71083ea7c4ac43d79b1ddaed76f3155e': Node(id='71083ea7c4ac43d79b1ddaed76f3155e', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11df09290>, search_kwargs={}), metadata=None),
 '969fbe7f2e9747779549777dd7a610b5': Node(id='969fbe7f2e9747779549777dd7a610b5', name='format_docs', data=RunnableLambda(format_docs), metadata=None),
 '06a46436ab8d45b1a33230b197c74f84': Node(id='06a46436ab8d45b1a33230b197c74f84', name='Passthrough', data=Runnab

In [9]:
retrieval_chain.get_graph().edges

[Edge(source='dc53538f6e85431fa42f64014611f63c', target='40766f0cb25e491488c3d3f28f1b5abf', data=None, conditional=False),
 Edge(source='04e21109430b4473bab6e92be87d6577', target='dc53538f6e85431fa42f64014611f63c', data=None, conditional=False),
 Edge(source='40766f0cb25e491488c3d3f28f1b5abf', target='d2e47cebe2324917a15bb6466a592773', data=None, conditional=False),
 Edge(source='04e21109430b4473bab6e92be87d6577', target='3d122ccf7c5a43cca7b7c7a08e87e5ab', data=None, conditional=False),
 Edge(source='3d122ccf7c5a43cca7b7c7a08e87e5ab', target='d2e47cebe2324917a15bb6466a592773', data=None, conditional=False),
 Edge(source='d2e47cebe2324917a15bb6466a592773', target='7e404e4b2ce4489194b773804c6b2438', data=None, conditional=False),
 Edge(source='7e404e4b2ce4489194b773804c6b2438', target='68ff55bbc2f94399b88cb9ff30c50833', data=None, conditional=False),
 Edge(source='9ed46daf135d472b9a81fecbb96886ff', target='8a3990fc21844ccdbb552db562f00700', data=None, conditional=False),
 Edge(source='68

# 그래프 출력

In [10]:
retrieval_chain.get_graph().print_ascii()

            +---------------------------------+          
            | Parallel<context,question>Input |          
            +---------------------------------+          
                    ***               ***                
                 ***                     ***             
               **                           ***          
+----------------------+                       **        
| VectorStoreRetriever |                        *        
+----------------------+                        *        
            *                                   *        
            *                                   *        
            *                                   *        
    +-------------+                     +-------------+  
    | format_docs |                     | Passthrough |  
    +-------------+*                    +-------------+  
                    ***               ***                
                       ***         ***                   
              

# 프롬프트 확인

In [11]:
retrieval_chain.get_prompts()

[PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer the question based only on the following context: {context}\nQuestion: {question}\n')]

-----
* End of Document *